In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
import numpy as np
from sklearn import metrics

In [ ]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
import json
import os

from pprint import pprint

In [ ]:
# from data.coco.coco import CocoGenerator as CocoGeneratorOld
from data.coco.coco2 import CocoGenerator

In [ ]:
from model.callbacks.metric_callbacks import MAPCallback
from model.callbacks.save_callback import SaveModel
from model.callbacks.scheduler import lr_scheduler

In [ ]:
from model.losses import BCE, PartialBCE

In [ ]:
from config import config_utils
from config.config import cfg
from experiments.launch_utils import parse_options_file

In [ ]:
p = 100

In [ ]:
parse_options_file('/home/caleml/partial-labels/config/pv_baseline50_sgd_448lrs.yaml')

# datasets

In [ ]:
# train
data_dir = '/share/DEEPLEARNING/datasets/mscoco'
batch_size = 16
# train_dataset = CocoGeneratorOld('train', data_dir, p=prop)

In [ ]:
generator_train = train_dataset.flow(batch_size=2)
X_train, Y_train = next(generator_train)

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
# validation
val_dataset = CocoGenerator('val', '/share/DEEPLEARNING/datasets/mscoco')

In [ ]:
print(len(val_dataset.image_ids_in_subset))

In [ ]:
batch_size = len(val_dataset.image_ids_in_subset)
generator_val = val_dataset.flow(batch_size=batch_size)
X_test, Y_test = next(generator_val)
print(X_test.shape, Y_test.shape)

### New dataset

In [ ]:
train_dataset = CocoGenerator(data_dir, batch_size, 'train', x_keys=['image'], y_keys=['multilabel'], year='2017', p=p)
print(train_dataset.nb_samples)

In [ ]:
x, y = train_dataset[0]

In [ ]:
print(type(x), type(x))
print(np.array(x).shape, len(x))
print(np.array(y).shape, len(y))

In [ ]:
# val dataset
val_dataset = CocoGenerator(data_dir, 'all', 'val', x_keys=['image'], y_keys=['multilabel'], year='2017')

In [ ]:
val_dataset.nb_samples

In [ ]:
X_test, Y_test = val_dataset[0]

In [ ]:
print(type(X_test), type(Y_test))
print(np.array(X_test).shape, len(X_test))
print(np.array(Y_test).shape, len(Y_test))

# model

In [ ]:
# dropping the Dense(1000)
model = ResNet50(include_top=True, weights='imagenet')
x = model.layers[-2].output
x = Dense(train_dataset.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.input, outputs=x)

In [ ]:
model.summary()

In [ ]:
lr = 0.1
loss = BCE()
# loss = PartialBCE(prop / 100)
model.compile(loss=loss, optimizer=SGD(lr=lr), metrics=['binary_accuracy'])

### Baseline model

In [ ]:
exp_folder = '/home/caleml/partial_experiments/exp_20190802_1803_TESTNB'
os.makedirs(exp_folder, exist_ok=True)

In [ ]:
from model.networks.baseline import Baseline
# from model.callbacks.prior_callback import PriorCallback

In [ ]:
model = Baseline(exp_folder, train_dataset.nb_classes, p)
model.load_config('coco_baseline50_sgd_448lrs')
model.build()

In [ ]:
# callbacks
cb_list = list()

map_cb = MAPCallback(X_test, Y_test, exp_folder, p)
cb_list.append(map_cb)

# Save Model
cb_list.append(SaveModel(exp_folder, p))


In [ ]:
# train
steps_per_epoch = len(train_dataset)
model.train(train_dataset, steps_per_epoch=steps_per_epoch, cb_list=cb_list)

# train

In [ ]:
# callbacks
exp_folder = '/home/caleml/partial_experiments/exp_20190723_0050_TESTNB'
os.makedirs(exp_folder, exist_ok=True)

cb_list = list()
cb_list.append(SaveModel(exp_folder, 100))

map_cb = MAPCallback(X_test, Y_test, exp_folder, prop=prop)
cb_list.append(map_cb)

cb_list.append(LearningRateScheduler(lr_scheduler))

In [ ]:
batch_size=32
nb_epochs=20

steps_per_epoch_train = int(len(train_dataset.id_to_label) / batch_size) + 1
model.fit_generator(train_dataset.flow(batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch_train,
                    epochs=nb_epochs,
                    callbacks=cb_list,
                    verbose=1)